# Solution: Measuring SeHCAT retention

This is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner

First version: 17th of September 2024

Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

The data sets were provided by the University Clinic for Radiology and Nuclear Medicine Magdeburg.

## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!wget -q -O - https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp/archive/refs/heads/main.tar.gz | tar -xzf - --strip-components=2 python-bootcamp-main/projects

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'projects')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl os numpy pydicom matplotlib

In [1]:
# Import packages
import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt

## --- Start notebook ---

### 0. Load folder/dataset:

In [ ]:
pat_folder = 'files/SehCAT/Pat1/'
pat_data = os.listdir(pat_folder)
print(pat_data)

### 1. Load DICOM data:

In [ ]:
# Read the DICOM file
dcm_file = pydicom.dcmread(pat_folder + pat_data[3])

# Print specific DICOM tags
print(dcm_file[0x0054, 0x0400].value)
print(dcm_file[0x0010, 0x1010].value)
print(dcm_file[0x0010, 0x1020].value)
print(dcm_file[0x0011, 0x100d].value)
print(dcm_file[0x0011, 0x1012].value)

# Get the pixel data
dcm_pixel_data = dcm_file.pixel_array

# Print the shape of the pixel data
print(dcm_pixel_data.shape)

### 2. Plot DICOMs

In [ ]:
# Plot the pixel data
fig, axs = plt.subplots(1, 4, figsize=(16, 5))

# Subplots with subtitles
axs[0].imshow(dcm_pixel_data[0], cmap='gray')
axs[0].set_title('ANT_0d_EM1')

axs[1].imshow(dcm_pixel_data[1], cmap='gray')
axs[1].set_title('POST_0d_EM1')

axs[2].imshow(dcm_pixel_data[2], cmap='gray')
axs[2].set_title('ANT_0d_EM2')

axs[3].imshow(dcm_pixel_data[3], cmap='gray')
axs[3].set_title('POST_0d_EM2')

plt.tight_layout()
plt.show()

### 3. Calculate the Retention

In [ ]:
# Define the decay factor
decay_factor = 1.04

# Read the DICOM files and get the pixel data
bg_0 = pydicom.dcmread(pat_folder + pat_data[0]).pixel_array
bg_7 = pydicom.dcmread(pat_folder + pat_data[1]).pixel_array
wb_0 = pydicom.dcmread(pat_folder + pat_data[2]).pixel_array
wb_7 = pydicom.dcmread(pat_folder + pat_data[3]).pixel_array

# Calculate the background and counts for the 0d and 7d images in the anterior and posterior views
background_0d_ant = np.sum(bg_0[0]) / 1000
background_0d_post = np.sum(bg_0[1]) / 1000
background_7d_ant = np.sum(bg_7[0]) / 1000
background_7d_post = np.sum(bg_7[1]) / 1000
ant_counts_0d = np.sum(wb_0[0]) / 1000
post_counts_0d = np.sum(wb_0[1]) / 1000
ant_counts_7d = np.sum(wb_7[0]) / 1000
post_counts_7d = np.sum(wb_7[1]) / 1000

# Calculate the retention after 1 week
retention_1w = decay_factor * (np.sqrt((ant_counts_7d - background_7d_ant)*(post_counts_7d - background_7d_post)) \
                               / np.sqrt((ant_counts_0d - background_0d_ant)*(post_counts_0d - background_0d_post))) * 100.

print(f"Retention after 1 week: {retention_1w}%")

#### 4. Function

In [ ]:
def calculate_retention(pat_folder, decay_factor=1.04):
    pat_data = os.listdir(pat_folder)

    # Read DICOM files
    bg_0 = pydicom.dcmread(pat_folder + pat_data[0]).pixel_array
    bg_7 = pydicom.dcmread(pat_folder + pat_data[1]).pixel_array
    wb_0 = pydicom.dcmread(pat_folder + pat_data[2]).pixel_array
    wb_7 = pydicom.dcmread(pat_folder + pat_data[3]).pixel_array

    # Calculate background and whole-body counts
    background_0d_ant = np.sum(bg_0[0]) / 1000
    background_0d_post = np.sum(bg_0[1]) / 1000
    background_7d_ant = np.sum(bg_7[0]) / 1000
    background_7d_post = np.sum(bg_7[1]) / 1000
    ant_counts_0d = np.sum(wb_0[0]) / 1000
    post_counts_0d = np.sum(wb_0[1]) / 1000
    ant_counts_7d = np.sum(wb_7[0]) / 1000
    post_counts_7d = np.sum(wb_7[1]) / 1000

    # Calculate retention
    retention_1w = round(decay_factor * (np.sqrt((ant_counts_7d - background_7d_ant)*(post_counts_7d - background_7d_post)) \
                                         / np.sqrt((ant_counts_0d - background_0d_ant)*(post_counts_0d - background_0d_post))) * 100., 2)
    return retention_1w

pat_folder = "files/SehCAT/Pat2/"
retention_1w = calculate_retention(pat_folder)
print(f"Retention after 1 week: {retention_1w}%")